# AUTOGRAD: 자동 미분

### `autograd` 패키지
- Tensor의 모든 연산에 대해 자동 미분을 제공
- 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의


### `torch.Tensor` 클래스
- `.requires_grad=True`로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)
- 모든 계산이 완료된 후 `.backward()` 를 호출하여 모든 gradient를 자동으로 계산
- Tensor의 gradient는 `.grad` 속성에 누적됨
- Tensor가 연산을 추적하는 것을 중단하게 하려면, `.detach()` 를 호출
- 코드 블럭을 `with torch.no_grad()`으로 감싸면, 기록을 추적하는 것(과 메모리를 사용하는 것)을 방지할 수 있음
    - 특히 `gradient`를 추적할 필요가 없는 상황인 모델을 평가(evaluate)할 때 유용


### `Function` 클래스
- gradient를 계산하기 위한 연산을 정의
    - 각 tensor는 `.grad_fn` 속성을 갖고 있는데, 이는 Tensor를 생성한 Function 을 참조 (단, 사용자가 만든 Tensor는 예외 : 이때 `grad_fn` 은 `None`)
- 도함수를 계산하기 위해서는 `Tensor` 의 `.backward()`를 호출하면 됨
    - `Tensor` 가 스칼라(scalar)인 경우(예. 하나의 요소 값만 갖는 등)에는 `backward`에 인자를 정해주지 않아도 됨
    - `Tensor` 가 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 `gradient` 인자로 전달해야 함

# Gradient 추적

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [2]:
# 기본 tensor 생성 시, requires_grad는 False
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False

# .requires_grad_(True)로 requires_grad값을 inplace로 변경 가능
a.requires_grad_(True)
print(a.requires_grad)

# 이제 연산을 추적하므로, grad_fn이 생겼음(Summation)
b = (a * a).sum()
print(b.grad_fn)

False
True


# Gradient 계산

#### `Tensor`가 스칼라(scalar)인 경우
- `backward()`에 인자 전달 안해도 됨

In [3]:
# requires_grad = True로 연산 추적
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
# grad_fn이 Addition
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
# grad_fn이 Multiplication
z = y * y * 3
print(z)
print(z.grad_fn)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


In [6]:
# grad_fn이 Mean
out = z.mean()
print(out)
print(out.grad_fn)

tensor(27., grad_fn=<MeanBackward0>)


In [7]:
# out은 scalar이므로, backward()에 인자 전달 안해도 됨

# out.backward()는 out.backward(torch.tensor(1., dtype=torch.float))와 동일
# 또한, 이는 구해지는 gradient에 1을 곱한것과 결과가 같음
# out.backward()
out.backward(torch.tensor(1., dtype=torch.float))

# `.grad`로 output에 대한 해당 변수의 gradient를 구할 수 있음
print(x.grad) # d(out)/dx : Jacobian Matrix(vector간 gradient라서)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [8]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

# gradient에 1을 곱한것과 결과가 같음 (위 자코비안에 2를 곱한 결과)
out.backward(torch.tensor(2., dtype=torch.float))
print(x.grad)

tensor([[9., 9.],
        [9., 9.]])


#### `Tensor`가 여러 개의 요소(vector)인 경우

- `backward()`의 `gradient` 인자로 vector를 전달해야 함

In [9]:
x = torch.randn(3, requires_grad=True)

out = x * 2
while out.data.norm() < 1000:
    out = out * 2

# out은 vector 형태(앞의 예제와 달리, scalar가 아님)
# 크기는 3x1
print(out)

tensor([ 484.8318,  173.9296, 1075.4950], grad_fn=<MulBackward0>)


In [10]:
# 3x1 크기의 vector를 gradient 인자로 전달해줘야 함
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
out.backward(gradient=v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


# 추가) CS231n 4강의 gradient 예제

<img src="./assets/cs231n_lecture_4_backprop_vectorized_form.png" alt="cs231n_lecture_4_backprop_vectorized_form" width="80%">

In [11]:
W = torch.tensor([[0.1, 0.5], [-0.3, 0.8]], dtype=torch.float, requires_grad=True)
x = torch.tensor([[0.2], [0.4]], dtype=torch.float, requires_grad=True)
print(W)
print(x)

tensor([[ 0.1000,  0.5000],
        [-0.3000,  0.8000]], requires_grad=True)
tensor([[0.2000],
        [0.4000]], requires_grad=True)


In [12]:
z = torch.matmul(W, x)
z.retain_grad() # z는 말단 노드가 아니라서, .retain_grad()를 호출해주어야 gradient를 보존할 수 있음
print(z)

tensor([[0.2200],
        [0.2600]], grad_fn=<MmBackward>)


In [13]:
out = z.norm() ** 2
print(out)

tensor(0.1160, grad_fn=<PowBackward0>)


In [14]:
out.backward()

In [15]:
# CS231n ppt에서와 동일한 결과를 확인할 수 있음
print(W.grad)
print(x.grad)
print(z.grad) # z.retain_grad() 안해주면 출력이 안됨

tensor([[0.0880, 0.1760],
        [0.1040, 0.2080]])
tensor([[-0.1120],
        [ 0.6360]])
tensor([[0.4400],
        [0.5200]])


# Gradient 추적 없이 코드블럭 감싸기

- `with torch.no_grad()`
- 모델을 평가(evaluate)할 때 유용

In [16]:
print(x.requires_grad)
print((x ** 2).requires_grad)

# 생성되는 tensor에 대해서만 추적을 하지 않음
# 따라서, evaluation 과정에서 계산한 output에 대한 gradient 추적만을 제한 가능
# (학습한 모델의 전체 파라미터에 대한 gradient 추적은 멈추지 않고,
#  prediction한 결과에 대해서만 추적 하지 않을 수 있음)
with torch.no_grad():
    print(x.requires_grad) # True
    print((x ** 2).requires_grad) # False    

True
True
True
False


In [17]:
print(x.requires_grad)

# .detach() 호출 시, requires_grad가 False가 됨
y = x.detach()
print(y.requires_grad)

True
False


In [18]:
# .detach()한 데이터와 tensor의 데이터는 같음
print(x.eq(y))
print(x.eq(y).all())

tensor([[True],
        [True]])
tensor(True)
